<a href="https://colab.research.google.com/github/valedalama/valedalama.github.io/blob/master/farmOS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparing for farmOS migration from v1.x to v2
a Jupyter Lab Notebook, prepared & hosted in Google Collaboratory, to document the process, results, next steps...



## Goals of this document
Two aspects of this migration are addressed here:
* Validating integrity of migrated data (v1.x -> v2) of farmOS; 
* Testing the farmOS API of v2, using farmOS.py library as interface

Those two issues can proceed in parallel -the sooner the better- so this analysis.  Other issues that must be addressed, but only when preconditions are met, are:
* WFS module (for serving geospatial data): awaits the next iteration of our v2 test instance (which awaits completion of the data migration test).
* Crop Planning module: not until some time after v2 is in production. Until then: we continue managing that data in spreadsheets.

## Context
https://farmos.discourse.group/t/farmos-2-x-releases-and-what-to-expect/736 <p>[link text](https://)
https://farmos.discourse.group/t/looking-for-farmos-2-x-migration-testers/575/8


## Validating Data Integrity, Step 1: working with CSVs

In [ ]:
# import the software tools we're gonna need, and get version info (for reproducibility)
import pandas as pd
print("Pandas version ",pd.__version__)

Pandas version  1.3.0


### Compare CSV output of report: all Assets of type Plant(ing, in v2.x)

In [ ]:
# load the full table of assets/ plantings (unfiltered) from v1.x, as of Wednesday, 21 July 2021 at 09:17
df=pd.read_csv("./farm_plantings-field_farm_move_to_tid_All-field_farm_group_target_id_All-2021-07-21T09-17-26.csv")
# do hi-level overview
print("Number of Rows, Columns = ",df.shape)  #to know the depth & scope of dataframe
print("Column Headers: ",df.columns)  # to see names of column headers
print("Datatype of index = ",type(df.index[0]))  # to confirm datatype of the index
# NB: number of rows (148) is correct; matches the report in UI

Number of Rows, Columns =  (148, 11)
Column Headers:  Index(['Farm asset', 'Asset ID', 'Unnamed: 2', 'Name', 'Crop/variety',
       'Season', 'Description', 'Flags', 'Group', 'Location', 'Archived'],
      dtype='object')
Datatype of index =  <class 'int'>


Compare "Column Headers" above with those displayed in farmOS UI:  
*Asset ID*.*Name*,Crop/variety,Season,**Description**,Flags,Group,Location,**Archived**

In [ ]:
#@title
# load the full table of assets/ plantings (unfiltered) from v2, as of Wednesday, 21 July 2021 at 09:22
df=pd.read_csv("./assets2.csv")
# do hi-level overview
print("Number of Rows, Columns = ",df.shape)  #to know the depth & scope of dataframe
print("Column Headers: ",df.columns)  # to see names of column headers
print("Datatype of index = ",type(df.index[0]))  # to confirm datatype of the index
# NB: number of rows (197) is INCORRECT, according to report in UI (just 145 records)

Number of Rows, Columns =  (197, 10)
Column Headers:  Index(['Bulk update', 'image_target_id', 'ID', 'Asset name', 'Asset type',
       'Flags', 'Parents', 'Group', 'Location', 'Status'],
      dtype='object')
Datatype of index =  <class 'int'>


Compare "Column Headers" above with those displayed in farmOS UI:  
*ID*,*Asset name*,Crop/variety,Season,Flags,**Parents**,Group,Location,**Status**



**Note**: there are some obvious discrepancies here, as explored in [this Discourse thread](https://farmos.discourse.group/t/looking-for-farmos-2-x-migration-testers/575/8?u=walt), including most notably the row count (which should be the same in both v1.x and v2) and column names. The row count discrepancy is a bug that developer Mike Stenta has [flagged as such](https://www.drupal.org/project/farm/issues/3224663).  As to column name discrepancies: given the scope of this upgrade, significant change to db schema were anticipated as a given, so that difference of 49 in the row count is no surprise.  What is more surprising is the difference in schema of same instance(s), comparing CSV output to view of the report as prepared in UI. Moreover, there are columns in the databaae that are filtered out of both views, though potentially important to the user, such as dates of record creation / modification, and username associated with those editing events.

### Compare CSV output of subset report: Plant(ings) in Location: EiraField orchard

In [ ]:
# load the .csv datafile from the v1.x instance, as of: Tuesday, 20 July 2021 at 19:36
df=pd.read_csv("./farm_plantings-name_EiraField-field_farm_move_to_tid_All-field_farm_group_target_id_All-2021-07-20T19-36-08.csv")
# do hi-level overview
print("Number of Rows, Columns = ",df.shape)  #to know the depth & scope of dataframe
print("Column Headers: ",df.columns)  # to see names of column headers
print("Datatype of index = ",type(df.index[0]))  # to confirm datatype of the index

Number of Rows, Columns =  (51, 11)
Column Headers:  Index(['Farm asset', 'Asset ID', 'Unnamed: 2', 'Name', 'Crop/variety',
       'Season', 'Description', 'Flags', 'Group', 'Location', 'Archived'],
      dtype='object')
Datatype of index =  <class 'int'>


In [ ]:
df.tail()  # eyeball the last 5 records

,Farm asset,Asset ID,Unnamed: 2,Name,Crop/variety,Season,Description,Flags,Group,Location,Archived
46,NaN,68,NaN,2020 EiraField Plums AutumnGiant,Plums AutumnGiant,2020,NaN,NaN,NaN,NaN,NaN
47,NaN,70,NaN,2020 EiraField Plums BlackGold,Plums BlackGold,2020,NaN,NaN,NaN,NaN,NaN
48,NaN,69,NaN,2020 EiraField Plums BlackStar,Plums BlackStar,2020,NaN,NaN,NaN,NaN,NaN
49,NaN,53,NaN,2020 EiraField Plums Metheley,Plums Metheley,2020,NaN,NaN,NaN,NaN,NaN
50,NaN,50,NaN,2020 EiraField Plums SãoBras,Plums SãoBras,2020,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.head()  #eyball the first 5 records

,Farm asset,Asset ID,Unnamed: 2,Name,Crop/variety,Season,Description,Flags,Group,Location,Archived
0,NaN,23,NaN,2019 EiraField Apples Anna,Apples Anna,2019,"15 Trees planted in bed #1, from north end on ...",NaN,NaN,EiraField,NaN
1,NaN,24,NaN,2019 EiraField Apples Jersey Mac,Apples Jersey Mac,2019,Planted 2 at S. end of Bed #2 on 2019.02.21,NaN,NaN,EiraField,NaN
2,NaN,27,NaN,2019 EiraField Apples Fuji,Apples Fuji,2019,NaN,NaN,NaN,EiraField,NaN
3,NaN,25,NaN,2019 EiraField Apples Golden Dorsett,Apples Golden Dorsett,2019,NaN,NaN,NaN,EiraField,NaN
4,NaN,29,NaN,2019 EiraField Apples Granny Smith,Apples Granny Smith,2019,NaN,NaN,NaN,EiraField,NaN


In [ ]:
# load  #the .csv datafile from v2 instance, as of: Tuesday, 20 July 2021 at 17:04
df=pd.read_csv("./assets.csv")
# do hi-level overview
print("Number of Rows, Columns = ",df.shape)  #to know the depth & scope of dataframe
print("Column Headers: ",df.columns)  # to see names of column headers
print("Datatype of index = ",type(df.index[0]))  # to confirm datatype of the index

Number of Rows, Columns =  (52, 10)
Column Headers:  Index(['Bulk update', 'image_target_id', 'ID', 'Asset name', 'Asset type',
       'Flags', 'Parents', 'Group', 'Location', 'Status'],
      dtype='object')
Datatype of index =  <class 'int'>


In [ ]:
df.tail()   # eyeball the last 5 records

,Bulk update,image_target_id,ID,Asset name,Asset type,Flags,Parents,Group,Location,Status
47,NaN,NaN,72,2020 EiraField Apricots Mogador,Plant,NaN,NaN,NaN,NaN,Active
48,NaN,NaN,73,2020 EiraField Apricots Mogador,Plant,NaN,NaN,NaN,NaN,Active
49,NaN,NaN,74,2020 EiraField Apricots Mogador,Plant,NaN,NaN,NaN,NaN,Active
50,NaN,NaN,75,2020 EiraField Apricots Mogador,Plant,NaN,NaN,NaN,NaN,Active
51,NaN,NaN,76,2020 EiraField Apricots TBD?,Plant,NaN,NaN,NaN,NaN,Active


In [ ]:
df.head()  #eyball the first 5 records

,Bulk update,image_target_id,ID,Asset name,Asset type,Flags,Parents,Group,Location,Status
0,NaN,NaN,187,EiraField,Land,NaN,18U,NaN,NaN,Active
1,NaN,NaN,23,2019 EiraField Apples Anna,Plant,NaN,NaN,NaN,EiraField,Active
2,NaN,NaN,24,2019 EiraField Apples Jersey Mac,Plant,NaN,NaN,NaN,EiraField,Active
3,NaN,NaN,25,2019 EiraField Apples Golden Dorsett,Plant,NaN,NaN,NaN,EiraField,Active
4,NaN,NaN,26,2019 EiraField Apples Royal Gala Riscada,Plant,NaN,NaN,NaN,EiraField,Active


## Testing the API

Easiest way to get a quick view of API endpoints is via [this JSON in the browser](https://vdl.farmos.dev/api/) -Firefox, that is (am unable to get human-readable view in Chrome or Safari).  From there, you can get a link to your desired endpoint... But navigating that tree to dig out what you want is not so easy.

Easier, if you have some facility with Python language (built into this notebook, as it happens) would be [this farmOS.py library](https://github.com/farmOS/farmOS.py) from Paul Weidner; that's what we'll be using in this section.

In [ ]:
from farmOS import farmOS #load the library
farm_client = farmOS(hostname="https://vdl.farmos.dev", client_id="farm", scope="farm_manager", version=2) #login
current_token = farm_client.authorize(username="apiTest", password="api+35+erAPI") #get OAuth token
farm_client.info() #poll API for endpoints

{'jsonapi': {'version': '1.0',
  'meta': {'links': {'self': {'href': 'http://jsonapi.org/format/1.0/'}}}},
 'data': [],
 'meta': {'links': {'me': {'meta': {'id': '65d791fe-8e35-488b-a262-e63ede764c49'},
    'href': 'http://vdl.farmos.dev/api/user/user/65d791fe-8e35-488b-a262-e63ede764c49'}},
  'farm': {'name': 'Quinta Vale da Lama',
   'url': 'http://vdl.farmos.dev',
   'version': '2.x',
   'system_of_measurement': 'metric'}},
 'links': {'asset--animal': {'href': 'http://vdl.farmos.dev/api/asset/animal'},
  'asset--equipment': {'href': 'http://vdl.farmos.dev/api/asset/equipment'},
  'asset--group': {'href': 'http://vdl.farmos.dev/api/asset/group'},
  'asset--land': {'href': 'http://vdl.farmos.dev/api/asset/land'},
  'asset--plant': {'href': 'http://vdl.farmos.dev/api/asset/plant'},
  'asset--sensor': {'href': 'http://vdl.farmos.dev/api/asset/sensor'},
  'asset--structure': {'href': 'http://vdl.farmos.dev/api/asset/structure'},
  'asset_type--asset_type': {'href': 'http://vdl.farmos.dev

In [ ]:
# logs = list(farm_client.log.iterate('observation'))
observation = farm_client.log.get('observation')
observation

{'jsonapi': {'version': '1.0',
  'meta': {'links': {'self': {'href': 'http://jsonapi.org/format/1.0/'}}}},
 'data': [{'type': 'log--observation',
   'id': '8939bf06-844d-4f73-b649-cbe92b732fcc',
   'links': {'self': {'href': 'http://vdl.farmos.dev/api/log/observation/8939bf06-844d-4f73-b649-cbe92b732fcc'}},
   'attributes': {'drupal_internal__id': 1,
    'drupal_internal__revision_id': 70,
    'langcode': 'en',
    'revision_created': '2021-06-23T00:43:35+00:00',
    'revision_log_message': 'Migrated from farmOS 1.x on 2021-06-22',
    'name': '5/9/2019 - 3:55:18 PM',
    'timestamp': '2019-05-06T23:00:00+00:00',
    'status': 'done',
    'created': '2019-05-09T14:58:46+00:00',
    'changed': '2019-05-09T14:58:46+00:00',
    'default_langcode': True,
    'revision_translation_affected': True,
    'data': None,
    'flag': [],
    'notes': {'value': 'Sheep jumped their South Slope paddock ',
     'format': 'default',
     'processed': '<p>Sheep jumped their South Slope paddock</p>\n'},


In [ ]:
len(observation['data'])

50

## Queries that don't work right yet